<a href="https://colab.research.google.com/github/BraulioDGM12/SSF-Oto25_GalvezBraulio/blob/main/SSF_Oto25G%C3%A1lvezBraulio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método de Horner y Cos (T3)

In [40]:
def poly_horner(A, x):
    p0 = A[-1]           # Empieza con el coeficiente de mayor grado (a_n)
    i = len(A) - 2      # Índice del siguiente coeficiente
    while i >= 0:
        p0 = p0 * x + A[i]  # Aplica Horner: p0 = (p0 * x) + siguiente coeficiente
        i -= 1            # Decimos que ahora i=i-1
    return p0              # Guarda p0


def poly_naive(A, x):
    p1 = 0
    for i, a in enumerate(A):
        p1 += (x ** i) * a
    return p1

def poly_iter(A, x):
    p2 = 0
    xn = 1
    for a in A:
        p2 += xn * a
        xn *= x
    return p2

A = [-19, 7, -4, 6]     #Definimos la lista de coeficientes
x=20         # Definimos la variable x
print(f"Para el método Horner el resultado es: {poly_horner(A, x)}")
print(f"Para el método Naive el resultado es: {poly_naive(A, x)}")
print(f"Para el método Iter el resultado es: {poly_iter(A, x)}")

Para el método Horner el resultado es: 46521
Para el método Naive el resultado es: 46521
Para el método Iter el resultado es: 46521


In [41]:
import math
import csv

# Serie directa
def cos_series_direct(x, tol=1e-8, CosenoReal=None):
    suma = 0
    n = 0
    term = 1
    results = []
    while abs(term) > tol:
        term = ((-1)**n) * (x**(2*n)) / math.factorial(2*n)
        suma += term
        error_rel = abs(suma - CosenoReal) / abs(CosenoReal)
        results.append((n, suma, error_rel))
        n += 1
    return results

# Serie parcial
def cos_series_partial(x, tol=1e-8, CosenoReal=None):
    suma = 0
    n = 0
    term = 1
    results = []
    while abs(term) > tol:
        suma += term
        error_rel = abs(suma - CosenoReal) / abs(CosenoReal)
        results.append((n, suma, error_rel))
        # siguiente término
        term *= - (x**2) / ((2*n+1)*(2*n+2))
        n += 1
    return results

# Función para guardar resultados en CSV
def guardar_csv(nombre, x, results, CosenoReal):
    with open(nombre, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["x", "Iteración", "Suma", "|Suma-cos(x)|/cos(x)"])
        for n, suma, error_rel in results:
            writer.writerow([x, n, f"{suma:.10f}", f"{abs(suma-CosenoReal)/CosenoReal:.10e}"])

# Ejemplo
x =90
CosenoReal = math.cos(x)

# Generar resultados
res_direct_1e4 = cos_series_direct(x, 1e-4, CosenoReal)
res_direct_1e8 = cos_series_direct(x, 1e-8, CosenoReal)
res_partial_1e4 = cos_series_partial(x, 1e-4, CosenoReal)
res_partial_1e8 = cos_series_partial(x, 1e-8, CosenoReal)

# Guardar CSVs
guardar_csv("serie_directa_tol1e-4.csv", x, res_direct_1e4, CosenoReal)
guardar_csv("serie_directa_tol1e-8.csv", x, res_direct_1e8, CosenoReal)
guardar_csv("serie_parcial_tol1e-4.csv", x, res_partial_1e4, CosenoReal)
guardar_csv("serie_parcial_tol1e-8.csv", x, res_partial_1e8, CosenoReal)

print("Archivos CSV generados, según yo") #Profesor, si está viendo esto y no encuentra los archivos en collab
# están en la parte izquierda donde está el símbolo de una carpeta. Supongo que ya lo sabe, pero por si las dudas.
print(f"El valor del cos({x}) es: {CosenoReal}")

Archivos CSV generados, según yo
El valor del cos(90) es: -0.4480736161291701
